### SEP a analizar 
<div>
<img src="SEP.png" width="500"/>
</div>


In [6]:
import pandapower as pp

#Crear red vacía
net = pp.create_empty_network()

#Crear barras
bus1 = pp.create_bus(net, vn_kv=220, name="Barra 1")
bus2 = pp.create_bus(net, vn_kv=110, name="Barra 2")
bus1A = pp.create_bus(net, vn_kv=110, name="Barra 1A")
bus2A = pp.create_bus(net, vn_kv=110, name="Barra 2A")
bus3A = pp.create_bus(net, vn_kv=110, name="Barra 3A")
bus1B = pp.create_bus(net, vn_kv=110, name="Barra 1B")
bus2B = pp.create_bus(net, vn_kv=110, name="Barra 2B")

#Crear barra slack en Barra 1
pp.create_ext_grid(net, bus1, vm_pu=1.0, va_degree=0.0, name="Slack")

#Crear transformador entre Barra 1 y Barra 2
pp.create_transformer(net, hv_bus=bus2, lv_bus=bus1, std_type="100 MVA 220/110 kV", name="Trafo 1-2")

#Crear líneas de transmisión
pp.create_line(net, from_bus=bus2, to_bus=bus1A, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 2-1A")
pp.create_line(net, from_bus=bus1A, to_bus=bus2A, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 1A-2A")
pp.create_line(net, from_bus=bus2A, to_bus=bus3A, length_km=20, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 2A-3A")
pp.create_line(net, from_bus=bus2, to_bus=bus1B, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 2-1B")
pp.create_line(net, from_bus=bus1B, to_bus=bus2B, length_km=30, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 1B-2B")
pp.create_line(net, from_bus=bus3A, to_bus=bus2B, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 3A-2B")

#Crear cargas
pp.create_load(net, bus2, p_mw=150, q_mvar=100, name="Load 2")  # Carga nominal en Barra 2
pp.create_load(net, bus1A, p_mw=30, q_mvar=20, name="Load 1A")
pp.create_load(net, bus2A, p_mw=52.5, q_mvar=35, name="Load 2A")
pp.create_load(net, bus3A, p_mw=22.5, q_mvar=15, name="Load 3A")
pp.create_load(net, bus2B, p_mw=90, q_mvar=60, name="Load 2B")
pp.create_load(net, bus1B, p_mw=15, q_mvar=10, name="Load 1B")



pp.runpp(net, tolerance_mva=1e-5)

# Mostrar resultados
print(net.res_bus)
print(net.res_line)
print(net.res_trafo)

      vm_pu  va_degree        p_mw       q_mvar
0  1.000000   0.000000 -441.342753 -2976.565858
1  0.615350 -47.272196  150.000000   100.000000
2  0.579759 -48.744698   30.000000    20.000000
3  0.539317 -50.606657   52.500000    35.000000
4  0.517404 -51.703177   22.500000    15.000000
5  0.586284 -48.462034   15.000000    10.000000
6  0.511160 -51.967794   90.000000    60.000000
    p_from_mw  q_from_mvar     p_to_mw  q_to_mvar     pl_mw   ql_mvar  \
0  127.959458    88.685107 -122.687432 -82.075858  5.272026  6.609249   
1   92.687432    62.075858  -88.088806 -57.063946  4.598626  5.011913   
2   35.588806    22.063946  -34.566986 -23.107935  1.021820 -1.043990   
3  104.461381    72.144302 -100.950680 -68.328551  3.510701  3.815750   
4   85.950680    58.328551  -78.037415 -50.171387  7.913265  8.157164   
5   12.066986     8.107935  -11.962585  -9.828613  0.104401 -1.720678   

   i_from_ka   i_to_ka      i_ka  vm_from_pu  va_from_degree  vm_to_pu  \
0   1.327940  1.336332  1.3363